# Introduction to MongoDB and `pymongo`

We will do some basic operations on a toy database (`students.json`).

## Import libraries: JSON, pymongo

In [1]:
import json
from pymongo import MongoClient

## Open and reset the database

In [2]:
client = MongoClient('localhost', 27017)
db = client.my_database
db.collection.drop()  # This clears the collection
print('db is a', type(db))
print('db.collection is a', type(db.collection))

db is a <class 'pymongo.database.Database'>
db.collection is a <class 'pymongo.collection.Collection'>


## Load the JSON file ("students.json") into our database

In [4]:
with open('students.json', 'r') as data_file:  # Remember to change the file path as needed
    data_json = json.load(data_file)
    db.collection.insert_many(data_json)

# Search queries

We can do queries using the function `find(query)`, where `query` must be a dictionary.

## Get all documents

To retrieve all documents in a collection, you can pass an empty query:

In [5]:
list(db.collection.find(
    {}
))

[{'_id': 1,
  'name': 'John',
  'age': 22,
  'city': 'Madrid',
  'sports': ['soccer', 'handball']},
 {'_id': 2, 'name': 'Steve', 'age': 24, 'city': 'Segovia'},
 {'_id': 3,
  'name': 'Marta',
  'age': 21,
  'city': 'Madrid',
  'sports': ['handball', 'basketball']}]

## Find documents by field

To filter by a specific key value, pass it in the `query` dictionary. Let's get all students that come from Madrid:

In [6]:
list(db.collection.find(
    {'city': 'Madrid'}
))

[{'_id': 1,
  'name': 'John',
  'age': 22,
  'city': 'Madrid',
  'sports': ['soccer', 'handball']},
 {'_id': 3,
  'name': 'Marta',
  'age': 21,
  'city': 'Madrid',
  'sports': ['handball', 'basketball']}]

## Counting results

To count how many documents our query returned, we can just use `len()` of the list of results. How many students come from Madrid?

In [7]:
len(list(db.collection.find(
    {'city': 'Madrid'}
)))

2

## Find documents with "AND" conditions

We simply put one key/value pair per condition:

In [8]:
list(db.collection.find(
    {'city': 'Madrid', 'age': 22}
))

[{'_id': 1,
  'name': 'John',
  'age': 22,
  'city': 'Madrid',
  'sports': ['soccer', 'handball']}]

## Find documents by subfields

In [9]:
list(db.collection.find(
    {'sports': 'soccer'}
))

[{'_id': 1,
  'name': 'John',
  'age': 22,
  'city': 'Madrid',
  'sports': ['soccer', 'handball']}]

## Range queries
    
Let's find all students older than 21. We use the operator `"$gt"` which means "greater than":

In [10]:
list(db.collection.find(
    {'age': {"$gt": 21}}
))

[{'_id': 1,
  'name': 'John',
  'age': 22,
  'city': 'Madrid',
  'sports': ['soccer', 'handball']},
 {'_id': 2, 'name': 'Steve', 'age': 24, 'city': 'Segovia'}]

## "Not equal" conditions

Which students do *not* like soccer?

We can find out using operator `$ne`:

In [11]:
list(db.collection.find(
    {'sports': {'$ne': 'soccer'}}
))

[{'_id': 2, 'name': 'Steve', 'age': 24, 'city': 'Segovia'},
 {'_id': 3,
  'name': 'Marta',
  'age': 21,
  'city': 'Madrid',
  'sports': ['handball', 'basketball']}]

## "Or" conditions

Let's find all students who are older than 21 *or* are from Madrid.

We need to use this syntax: `$or: [condition1, condition2]`:

In [12]:
list(db.collection.find(
    {'$or': [{'age': {"$gt": 21}}, {'city': 'Madrid'}]}
))

[{'_id': 1,
  'name': 'John',
  'age': 22,
  'city': 'Madrid',
  'sports': ['soccer', 'handball']},
 {'_id': 2, 'name': 'Steve', 'age': 24, 'city': 'Segovia'},
 {'_id': 3,
  'name': 'Marta',
  'age': 21,
  'city': 'Madrid',
  'sports': ['handball', 'basketball']}]

## Find documents where a field exists

You can use the operator `$exists` to select only those documents that have a specific field.

Let's find out who has information about the sports they like:

In [13]:
list(db.collection.find(
    { "sports" : { "$exists" : True } }
))

[{'_id': 1,
  'name': 'John',
  'age': 22,
  'city': 'Madrid',
  'sports': ['soccer', 'handball']},
 {'_id': 3,
  'name': 'Marta',
  'age': 21,
  'city': 'Madrid',
  'sports': ['handball', 'basketball']}]

# Inserting documents

To add one document, use `insert_one(document)`. Let's add a new student:

In [13]:
db.collection.insert_one(
    {'_id': 4, 'name': 'Peter'}
)

To add several documents in one go, use `insert_many(list of documents)`:

In [14]:
db.collection.insert_many([
    {'_id': 5, 'name': 'Emma'},
    {'_id': 6, 'name': 'Diego'}
])

InsertManyResult([5, 6], acknowledged=True)

# Updating documents

To update documents matching a condition, use `update_many(condition, information)`.

Let's record that Emma likes soccer:

In [15]:
db.collection.update_many(
    {'name': 'Emma'},
    {'$set': {'sports': ['soccer']}}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Let's check that it worked:

In [16]:
list(db.collection.find({'name': 'Emma'}))

[{'_id': 5, 'name': 'Emma', 'sports': ['soccer']}]

# Deleting documents

To delete documents with a certain condition, use `delete_many(condition)`. Let's delete the student named "Diego":

In [17]:
db.collection.delete_many(
    {'name': 'Diego'}
)

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

We can check that, indeed, that student is no longer in the database:

In [18]:
list(db.collection.find(
    {'name': 'Diego'}
))

[]

# Sorting

We can sort a query result by appending `.sort(field, order)`, where `order` should be 1 (for ascending order) or -1 (descending).

Let's sort all students by increasing:

In [22]:
list(db.collection.find().sort("age", 1))

[{'_id': 5, 'name': 'Emma', 'sports': ['soccer']},
 {'_id': 3,
  'name': 'Marta',
  'age': 21,
  'city': 'Madrid',
  'sports': ['handball', 'basketball']},
 {'_id': 1,
  'name': 'John',
  'age': 22,
  'city': 'Madrid',
  'sports': ['soccer', 'handball']},
 {'_id': 2, 'name': 'Steve', 'age': 24, 'city': 'Segovia'}]

# Exporting to JSON

If you want to save a query result into a JSON file, use `json.dump()` like this:

In [20]:
with open("students2.json", "w") as f:
    json.dump(list(db.collection.find()), f) 

# Aggregation queries

*Aggregations* summarize fields from many documents (for example, compute the sum or the average of a field).

This is done with the `$group` keyword. For example, let's find out the average age of all students:


In [21]:
list(db.collection.aggregate([
    {'$group': {'_id': 1, 'average_age': {'$avg': '$age'}}}
]))

[{'_id': 1, 'average_age': 22.333333333333332}]

## Aggregate with a condition

We can add a condition (`$match`) before `$group` to restrict our aggregation.

Let's find the maximum age of all students who come from Madrid:

In [22]:
list(db.collection.aggregate([
    {'$match': {'city': 'Madrid'}},
    {'$group': {'_id': 1, 'max_madrid_age': {'$max': '$age'}}}
]))

[{'_id': 1, 'max_madrid_age': 22}]